### 1、策略一：
    1、首先计算出 group2 中的每一个点与 group1 中哪一个点相连，能够是最小值，
    2、之后选择 group1 中与group2 中哪个点相连，能够有最小值，
    3、最后检查，在 group2 中有哪个点没有连接

In [10]:
from functools import lru_cache

class Solution:
    def connectTwoGroups(self, cost):
        @lru_cache(None)
        def dp(i, mask):
            # 如果连到了 group1 的最后一位时
            if i == size1:
                ans2 = 0
                for j in range(size2):
                    # 检查在group2中，还有哪些点没有相连
                    if mask & (1 << j) == 0: 
                        ans2 += min_cost2[j]
                return ans2
            ans = float('inf')
            # 尝试每一种 group1 与 group2 相连接的可能性，可能在最后的时候，会有 group2 中的点没有连接，此时使用min_cost2进行弥补
            for j in range(size2):
                ans = min(ans, dp(i+1, mask | (1 << j)) + cost[i][j])
            return ans
        
        size1, size2 = len(cost), len(cost[0])
        min_cost2 = [float('inf')] * size2
        for i in range(size2):
            for j in range(size1):
                min_cost2[i] = min(min_cost2[i], cost[j][i])
        res = dp(0, 0)
        return res

In [11]:
solution = Solution()
solution.connectTwoGroups(cost = [[15, 96], 
                                  [36, 2]])

17

### 1、策略二：
    1、首先计算 group 1 中 与 group 2 每个点相连的最小 cost
    2、之后计算 group 1 中下一个点与 group2 中相连的最小值，在这种情况下，有可能group 1 的下个点已经和前面的连接过的group2 的点相连
    3、这个时候就需要取最小值

In [67]:
class Solution:
    def connectTwoGroups(self, cost):
        size1, size2 = len(cost), len(cost[0])
        N = 1 << size2
        
        dp = [[float('inf')] * N for _ in range(size1)]
        # 初始化 dp[0] 与 group2 中每个点相连的 cost 值
        for state in range(1, N): # 遍历所有与 group2 中的点相连的可能情况
            total = 0
            for j in range(size2):
                if ((state >> j) & 1) == 1:
                    total += cost[0][j]
            dp[0][state] = total
        
        for i in range(1, size1):
            for state in range(1, N): # 需要遍历每一种 group 1 中的点与group2 中的点连接情况
                sub_state = state
                while sub_state:
                    sum_cost = 0
                    for j in range(size2):
                        if sub_state >> j & 1 == 1:
                            sum_cost += cost[i][j]
                    dp[i][state] = min(dp[i][state], dp[i-1][state-sub_state] + sum_cost)
                    sub_state = (sub_state - 1) & state  # 子集合
                
                min_cost = float('inf')
                for j in range(size2):
                    min_cost = min(min_cost, cost[i][j])
                dp[i][state] = min(dp[i][state], dp[i-1][state] + min_cost)
        print(dp)
        return dp[-1][-1]

In [68]:
solution = Solution()
solution.connectTwoGroups(cost = [[15, 96], 
                                  [36, 2]])

[[inf, 15, 96, 111], [inf, 17, 98, 17]]


17

In [ ]:
[[inf, 15, 96, 111], 
 [inf, inf, inf, inf]]

In [ ]:
class Solution {
    int dp[12][1<<12];
public:
    int connectTwoGroups(vector<vector<int>>& cost) 
    {
        int m = cost.size();
        int n = cost[0].size();

        dp[0][0] = INT_MAX/2;
        for (int state = 1; state < (1<<n); state++)
        {
            int sum = 0;
            for (int j=0; j<n; j++)
            {
                if (((state>>j)&1)==1)
                    sum += cost[0][j];
            }
            dp[0][state] = sum;
        }

        for (int i=1; i<m; i++)
        {
            dp[i][0] = INT_MAX/2;
            for (int state = 1; state < (1<<n); state++)
            {
                dp[i][state] = INT_MAX/2;
                for (int subset=state; subset>0; subset=(subset-1)&state)
                {
                    int sum = 0;
                    for (int j=0; j<n; j++)
                    {
                        if (((subset>>j)&1)==1)
                            sum += cost[i][j];
                    }
                    dp[i][state] = min(dp[i][state], dp[i-1][state-subset]+sum);
                }

                int minPath = INT_MAX;
                for (int j=0; j<n; j++)
                    minPath = min(minPath, cost[i][j]);
                dp[i][state] = min(dp[i][state], dp[i-1][state] + minPath);                                 
            }            
        }

        return dp[m-1][(1<<n)-1];
    }
};

In [16]:
# 寻找 state 的子集
state = 14
sub_state = state
while sub_state:
    print(bin(sub_state))
    sub_state = (sub_state - 1) & state

0b1110
0b1100
0b1010
0b1000
0b110
0b100
0b10
